In [183]:
import pandas as pd
import psycopg2
import pytz
import datetime
from transliterate import translit
import random

In [284]:
conn = psycopg2.connect(
    host="localhost",
    database="skyscanner",
    user="postgres",
    password="")
    
cur = conn.cursor()
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

('PostgreSQL 10.19, compiled by Visual C++ build 1800, 64-bit',)


In [184]:
from people_data import maleNames, maleSurNames, femaleNames, femaleSurNames, malePatronymics, femalePatronymics, phoneCodes

In [367]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()

In [369]:
from people_data import maleNames, maleSurNames, femaleNames, femaleSurNames, malePatronymics, femalePatronymics, phoneCodes

def randomBool():
    return bool(random.getrandbits(1))

def generatePerson(passenger_id: int, booking_id: int):
    global maleNames, maleSurNames, femaleNames, femaleSurNames, malePatronymics, femalePatronymics, phoneCodes
    if randomBool():
        name = random.choice(maleNames)
        surname = random.choice(maleSurNames)
        patronymic = random.choice(malePatronymics)
    else:
        name = random.choice(femaleNames)
        surname = random.choice(femaleSurNames)
        patronymic = random.choice(femalePatronymics)
    age = random.randint(18, 100)
    date_of_birth = datetime.date.today() - datetime.timedelta(days=365*age)
    citizenship = random.choice(['Россия'])
    document_type = 'паспорт'
    document_number = ''.join([str(random.randint(1, 9)) for _ in range(4)]) + ' ' + ''.join([str(random.randint(1, 9)) for _ in range(6)])
    _datetime = datetime.datetime.now(pytz.timezone('Europe/Moscow')) - datetime.timedelta(days=random.randint(1, 20), hours=random.randint(1, 22), minutes=random.randint(1, 55), seconds=random.randint(1, 55))
    phone = '+7' + random.choice(phoneCodes) \
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))\
        + str(random.randint(1, 9))
    eMail = (translit(name[0] + surname, 'ru', reversed=True) + "@yandex.ru").replace("'", "")
    return (passenger_id, date_of_birth, citizenship, document_type, document_number, surname, name, patronymic), \
    (booking_id, _datetime, phone, eMail, surname, name, patronymic) 

def getAvailableSeat(conn):
    cur = conn.cursor()
    _date = random.randint(18, 30)
    cur.execute(f'''
        WITH available_flights as (SELECT
        flight_id, departure_time, arrival_time, aircraft_id, airlines.name as airline
    FROM flights
    INNER JOIN airlines on flights.airline_id = airlines.airline_id
    WHERE
        status = 'available' and
        date(flights.departure_time) = date('2022-03-{_date}') and
        flights.departure_time > date('2022-03-{_date}') + '4 hour'::interval and
        flights.departure_airport_id IN (
            SELECT airport_id FROM airports, cities
            WHERE
                airports.city_id = cities.city_id and
                cities.name = 'Симферополь'
            ) and
        flights.arrive_airport_id IN (
            SELECT airport_id FROM airports, cities
            WHERE
                airports.city_id = cities.city_id and
                cities.name = 'Москва'
            ))
    SELECT DISTINCT
        flight_id, seats.number, seats.seat_id, price
    FROM available_flights, seats
    WHERE
        seats.aircraft_id = available_flights.aircraft_id and
        seats.status = 'available' and
        seats.category = 'Взрослый' and
        seats.class = 'Бизнес'
    ORDER BY seats.price ASC
    LIMIT 1;
    ''')
    return cur.fetchall()[0]

def bookSeat(conn, seat_id):
    cur = conn.cursor()
    cur.execute(
    f'''
    UPDATE seats
    SET status = 'booked'
    WHERE seat_id = {seat_id};
    ''')
    conn.commit()

def addTicketBooking(ticket, booking, passenger):
    cursor = conn.cursor()

    cursor.execute("INSERT INTO passengers (passenger_id, date_of_birth, citizenship, document_type, document_number, surname, name, patronymic) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", passenger)
    cursor.execute("INSERT INTO bookings (booking_id, datetime, phone, email, surname, name, patronymic) VALUES(%s, %s, %s, %s, %s, %s, %s)", booking)
    cursor.execute("INSERT INTO tickets (ticket_id, seat_number, passenger_id, flight_id, booking_id) VALUES(%s, %s, %s, %s, %s)", ticket)
    
    conn.commit()

def generateTicketandBookSeat(conn, id: int, passenger_id: int, booking_id: int):
    seat = getAvailableSeat(conn)
    flight_id = seat[0]
    seat_number = seat[1]
    seat_id = seat[2]
    bookSeat(conn, str(seat_id))
    return id, seat_number, passenger_id, flight_id, booking_id

In [353]:
bookSeat(conn, '1501340010055')

booked 1501340010055


In [356]:
getAvailableSeat(conn)

(150134008, '10A', 1501340080055, 3906.0)

In [372]:
for i in range(301, 302):
    passenger, booking = generatePerson(i, i)
    ticket = generateTicketandBookSeat(conn, id=i, passenger_id=passenger[0], booking_id=booking[0])
    addTicketBooking(ticket, booking, passenger)
    # print(passenger, booking, ticket)

(301, datetime.date(1936, 1, 7), 'Россия', 'паспорт', '1332 388335', 'Федоров', 'Евгений', 'Александрович')
